In [1]:
import random
import pandas as pd
from datetime import datetime

# Temporary workflow
# (1) Download csv with `heroku run make exportdb && make importdb`
# (2) save links to ~/Documents/links.txt in onetab format
# (3) run this script (or otherwise modify the CSV as desired)
# (4) check date cell
# (5) Upload back up with `make csv_to_s3 && heroku run make importdb`
links = pd.read_csv('data/export.csv')
links

/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_41309/1281081789.py:11: DtypeWarning: Columns (7,12) have mixed types. Specify dtype option on import or set low_memory=False.
  links = pd.read_csv('data/export.csv')


,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98286,160811,https://bestofecontwitter.substack.com/,Best of Econtwitter,\N,\N,2022-12-28 16:01:31.298875,2022-12-28 16:01:31.298886,\N,\N,Custom,71,0,NaN
98287,160812,https://twitter.com/DavidAFrench/status/160784...,https://twitter.com/DavidAFrench/status/160784...,\N,\N,2022-12-28 16:01:31.310915,2022-12-28 16:01:31.310926,\N,\N,Custom,4,0,NaN
98288,160813,https://twitter.com/emmalrichter/status/160778...,"Emma Richter on Twitter: ""Thread of the best t...",\N,\N,2022-12-28 16:01:31.321883,2022-12-28 16:01:31.321897,\N,\N,Custom,94,0,NaN
98289,160814,https://twitter.com/sama/status/16074721047270...,"Sam Altman on Twitter: ""haven’t seen this as a...",\N,\N,2022-12-28 16:01:31.335260,2022-12-28 16:01:31.335272,\N,\N,Custom,38,0,NaN


In [2]:
links['added'].max()

'2022-12-28 16:01:31.346476'

In [3]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]['id']

1299       1306
1300       1307
1301       1308
1302       1309
1303       1310
          ...  
98286    160811
98287    160812
98288    160813
98289    160814
98290    160815
Name: id, Length: 11732, dtype: int64

In [4]:
links[links['url'].duplicated()]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
98204,160729,https://www.centreforeffectivealtruism.org/blo...,CEA’s 2017 review and 2018 plans,\N,www.centreforeffectivealtruism.org,2022-12-28 14:10:20.675312+00,2022-12-28 14:10:34.140752+00,0.0,\N,CEA,86,0,\N
98238,160763,https://danluu.com/dunning-kruger/,Dunning-Kruger,\N,\N,2022-12-28 16:01:30.743022,2022-12-28 16:01:30.743033,\N,\N,Custom,28,0,NaN
98242,160767,https://www.palladiummag.com/2022/11/14/why-we...,Why We Need the Center for Strategic Translation,\N,\N,2022-12-28 16:01:30.784060,2022-12-28 16:01:30.784068,\N,\N,Custom,14,0,NaN
98243,160768,https://mobile.twitter.com/drorpoleg/status/14...,https://mobile.twitter.com/drorpoleg/status/14...,\N,\N,2022-12-28 16:01:30.796385,2022-12-28 16:01:30.796418,\N,\N,Custom,43,0,NaN
98247,160772,https://twitter.com/stephsmithio/status/143393...,https://twitter.com/stephsmithio/status/143393...,\N,\N,2022-12-28 16:01:30.840779,2022-12-28 16:01:30.840791,\N,\N,Custom,50,0,NaN
98249,160774,https://twitter.com/brandonthezhang/status/145...,https://twitter.com/brandonthezhang/status/145...,\N,\N,2022-12-28 16:01:30.864244,2022-12-28 16:01:30.864256,\N,\N,Custom,83,0,NaN
98254,160779,https://danluu.com/productivity-velocity/,Some reasons to work on productivity and velocity,\N,\N,2022-12-28 16:01:30.918713,2022-12-28 16:01:30.918727,\N,\N,Custom,100,0,NaN
98255,160780,https://danluu.com/percentile-latency/,Goodhearting IQ cholesterol and tail latency,\N,\N,2022-12-28 16:01:30.930730,2022-12-28 16:01:30.930742,\N,\N,Custom,78,0,NaN
98258,160783,https://rootsofprogress.org/how-early-american...,How early American inventors funded their vent...,\N,\N,2022-12-28 16:01:30.965325,2022-12-28 16:01:30.965338,\N,\N,Custom,33,0,NaN


In [5]:
links[links['url'].duplicated()]['id']

98204    160729
98238    160763
98242    160767
98243    160768
98247    160772
98249    160774
98254    160779
98255    160780
98258    160783
Name: id, dtype: int64

In [6]:
link_file = open('/Users/peterhurford/Documents/links.txt', 'r')
new_links = link_file.readlines()

In [7]:
def new_entry(links, url, title):
    return {'id': links['id'].max() + 1,
            'url': str(url).strip().replace(',', ''),
            'title': str(title).strip().replace(',', ''),
            'summary': '\\N',
            'domain': '\\N',
            'added': str(datetime.now()),
            'modified': str(datetime.now()),
            'liked': '\\N',
            'category': '\\N',
            'aggregator': 'Custom',
            'seed': random.randint(1, 100),
            'tweet': 0}

for i, new_link in enumerate(new_links):
    if i % 10 == 0 or i == len(new_links) - 1:
        print('...{}/{}'.format(i + 1, len(new_links)))
    if ' | ' in new_link:
        new_link = new_link.replace('    - ', '').replace('\n', '')
        entry = new_entry(links, new_link.split(' | ')[0], new_link.split(' | ')[1])
        links = links.append(entry, ignore_index=True)

links

/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_41309/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)


...1/82
...11/82


/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_41309/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_41309/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_41309/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_41309/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

...21/82
...31/82


/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_41309/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_41309/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_41309/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_41309/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

...41/82
...51/82


/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_41309/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_41309/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_41309/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_41309/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

...61/82
...71/82
...81/82
...82/82


/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_41309/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_41309/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_41309/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  links = links.append(entry, ignore_index=True)
/var/folders/_q/gs93kbt10t3dq5qqyt50wcgc0000gn/T/ipykernel_41309/559676766.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat in

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98368,160893,https://bestofecontwitter.substack.com/,Best of Econtwitter,\N,\N,2022-12-28 16:01:55.183474,2022-12-28 16:01:55.183511,\N,\N,Custom,67,0,NaN
98369,160894,https://twitter.com/DavidAFrench/status/160784...,https://twitter.com/DavidAFrench/status/160784...,\N,\N,2022-12-28 16:01:55.195371,2022-12-28 16:01:55.195381,\N,\N,Custom,93,0,NaN
98370,160895,https://twitter.com/emmalrichter/status/160778...,"Emma Richter on Twitter: ""Thread of the best t...",\N,\N,2022-12-28 16:01:55.207825,2022-12-28 16:01:55.207868,\N,\N,Custom,93,0,NaN
98371,160896,https://twitter.com/sama/status/16074721047270...,"Sam Altman on Twitter: ""haven’t seen this as a...",\N,\N,2022-12-28 16:01:55.220861,2022-12-28 16:01:55.220873,\N,\N,Custom,51,0,NaN


In [8]:
links[(links['aggregator'] == 'Custom') & (links['liked'] != '1') & (links['liked'] != '0') & (links['liked'] != '-1')]

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
1299,1306,http://blog.beeminder.com/trust/,Trusting Your Divided Self | Beeminder Blog,\N,blog.beeminder.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,10,0,0
1300,1307,http://powlyglot.com/fi3m-review-part-1-focus/,"""What I Learned from Fluent in 3 Months\, Part...",\N,powlyglot.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,productivity,Custom,9,0,0
1301,1308,http://effective-altruism.com/ea/u8/ea_intervi...,"""EA Interview Series\, February 2016: John Sal...",\N,effective-altruism.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,effectivealtruism,Custom,85,0,0
1302,1309,https://fivethirtyeight.com/features/most-2020...,Most 2020 Candidates Have Something In Common:...,\N,fivethirtyeight.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,1.0,politicalscience,Custom,82,0,0
1303,1310,http://www.smartpassiveincome.com/backlinking-...,THE Backlinking Strategy That Works – 2014 and...,\N,www.smartpassiveincome.com,2020-01-01 05:00:00+00,2020-01-01 05:00:00+00,0.0,entrepreneurship,Custom,59,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98368,160893,https://bestofecontwitter.substack.com/,Best of Econtwitter,\N,\N,2022-12-28 16:01:55.183474,2022-12-28 16:01:55.183511,\N,\N,Custom,67,0,NaN
98369,160894,https://twitter.com/DavidAFrench/status/160784...,https://twitter.com/DavidAFrench/status/160784...,\N,\N,2022-12-28 16:01:55.195371,2022-12-28 16:01:55.195381,\N,\N,Custom,93,0,NaN
98370,160895,https://twitter.com/emmalrichter/status/160778...,"Emma Richter on Twitter: ""Thread of the best t...",\N,\N,2022-12-28 16:01:55.207825,2022-12-28 16:01:55.207868,\N,\N,Custom,93,0,NaN
98371,160896,https://twitter.com/sama/status/16074721047270...,"Sam Altman on Twitter: ""haven’t seen this as a...",\N,\N,2022-12-28 16:01:55.220861,2022-12-28 16:01:55.220873,\N,\N,Custom,51,0,NaN


In [9]:
def clean(txt):
    if not isinstance(txt, str):
        return txt
    txt = txt.replace('&nbsp;', '')
    txt = txt.replace('&ldquo;', '"')
    txt = txt.replace('&rdquo;', '"')
    txt = txt.replace('&lsquo;', '\'')
    txt = txt.replace('&rsquo;', '\'')
    txt = txt.replace('&mdash;', '-')
    txt = txt.replace('&ndash;', '-')
    for i in range(10):
        txt = txt.replace('"""', '"')
        txt = txt.replace('\'\'\'', '\'')
    txt = txt.replace('n"t', 'n\'t')
    txt = txt.replace('n""t', 'n\'t')
    return txt

links = links[links['id'].notnull()]   # Drop empty column
links['id'] = links['id'].astype(int)  # Fix float ID issue
links['seed'] = links['seed'].astype(int)
links['tweet'] = links['tweet'].apply(lambda x: 0 if str(x) == '\\N' else str(x).split('.')[0]).astype(int)
links['summary'] = links['summary'].apply(clean)
links['title'] = links['title'].apply(clean)
links = links.sort_values('id')
links

,id,url,title,summary,domain,added,modified,liked,category,aggregator,seed,tweet,starred
0,0,http://moralitylab.bc.edu/wp-content/uploads/2...,Moral Realism As Moral Motivation: The Impact ...,"""People disagree about whether 'moral facts' a...",moralitylab.bc.edu,2016-12-06 05:00:00+00,2016-12-06 05:00:00+00,1.0,ethics,custom,17,0,0
1,1,http://www.jefftk.com/news/2013-04-01,The Unintuitive Power Laws of Giving,"""Why give globally? Why give money? Why health...",www.jefftk.com,2016-12-07 05:00:00+00,2016-12-07 05:00:00+00,1.0,giving,custom,59,0,0
2,2,http://calnewport.com/blog/2013/03/24/how-to-w...,How to Write Six Important Papers a Year witho...,"""I'm fascinated by people who produce a large ...",calnewport.com,2016-12-08 05:00:00+00,2016-12-08 05:00:00+00,1.0,productivity,custom,56,0,0
3,3,http://www.freakonomics.com/2012/10/31/an-alte...,An Alternative to Democracy,"""In Glen's voting mechanism\, every voter can ...",www.freakonomics.com,2016-12-09 05:00:00+00,2016-12-09 05:00:00+00,1.0,politicalscience,custom,29,0,0
4,4,http://www.wired.com/vanish/2009/11/ff_vanish2...,Writer Evan Ratliff Tried to Vanish: Here's Wh...,"""August 13\, 6:40 PM: I'm driving East out of ...",www.wired.com,2016-12-10 05:00:00+00,2016-12-10 05:00:00+00,1.0,technology,custom,92,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98368,160893,https://bestofecontwitter.substack.com/,Best of Econtwitter,\N,\N,2022-12-28 16:01:55.183474,2022-12-28 16:01:55.183511,\N,\N,Custom,67,0,NaN
98369,160894,https://twitter.com/DavidAFrench/status/160784...,https://twitter.com/DavidAFrench/status/160784...,\N,\N,2022-12-28 16:01:55.195371,2022-12-28 16:01:55.195381,\N,\N,Custom,93,0,NaN
98370,160895,https://twitter.com/emmalrichter/status/160778...,"Emma Richter on Twitter: ""Thread of the best t...",\N,\N,2022-12-28 16:01:55.207825,2022-12-28 16:01:55.207868,\N,\N,Custom,93,0,NaN
98371,160896,https://twitter.com/sama/status/16074721047270...,"Sam Altman on Twitter: ""haven’t seen this as a...",\N,\N,2022-12-28 16:01:55.220861,2022-12-28 16:01:55.220873,\N,\N,Custom,51,0,NaN


In [10]:
links.to_csv('data/export.csv', index=False)